In [ ]:
import requests
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

url = "https://gist.githubusercontent.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223/raw/d72b9558a11523adbe13300b41321ecd93d331d3/document.txt"
response = requests.get(url)
response.raise_for_status()
text = response.text

document = Document(page_content=text, metadata={"source": url})

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents([document])

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vectorstore = FAISS.from_documents(docs, embeddings)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

llm = OpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0.1)
prompt = PromptTemplate(template="Use the following documents to answer the question: {documents}\nQuestion: {question}\nAnswer:", input_variables=["documents", "question"])

llm_chain = LLMChain(llm=llm, prompt=prompt)

stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="documents")

qa_chain = RetrievalQA(retriever=retriever, combine_documents_chain=stuff_chain, memory=memory)

questions = [
    "Is Aaronson guilty?",
    "What message did he write in the table?",
    "Who is Julia?"
]

for question in questions:
    result = qa_chain.run(question)
    print(f"Q: {question}\nA: {result}\n")

print("\nConversation History:")
print(memory.chat_history)